In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pylab as plt
import plotly.graph_objects as go
from typing import Tuple
import vectorbt as vbt

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

## Acknowledgement 

Credit to [kridtapon](https://medium.com/@kridtapon) and learning from his posts on Medium.

In [2]:
# Define the stock symbol and time period
symbol = 'SPY'
start_date = '2020-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [3]:
nrecent = 100
fig = go.Figure(data=[go.Candlestick(
    x=df.index[-nrecent:],
    open=df['Open'][-nrecent:],
    high=df['High'][-nrecent:],
    low=df['Low'][-nrecent:],
    close=df['Close'][-nrecent:]
)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    title=f'{symbol} Stock Price',
    title_x=0.5,  # Center the title
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly_dark'
)
fig.show()

## Gopalakrishnan Range Index (GAPO)

In [4]:
def gapo(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """
    Calculate the Gopalakrishnan Range Index (GAPO) for a given DataFrame.

    The GAPO measures market volatility based on the range of price movements over a specified period.

    Args:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' price columns.
        period (int, optional): The lookback period for calculating the GAPO. Defaults to 14.

    Returns:
        pd.DataFrame: DataFrame with added 'GAPO' column.
    """
    df = df.copy()
    df['range'] = df['High'] - df['Low']
    df['max_range'] = df['range'].rolling(window=period).max()
    df['GAPO'] = (df['range'] / df['max_range']) * 100
    return df


def gapo_signal(df: pd.DataFrame, threshold: float = 50) -> pd.DataFrame:
    """
    Generate trading signals based on the GAPO indicator.

    A buy signal (1) is generated when GAPO is below the lower threshold.
    A sell signal (-1) is generated when GAPO is above the upper threshold.
    A hold signal (0) is used otherwise.

    Args:
        df (pd.DataFrame): DataFrame containing the 'GAPO' column.
        threshold (float, optional): The mid-threshold value to determine buy/sell levels. Defaults to 50.

    Returns:
        pd.DataFrame: DataFrame with an added 'Signal' column (1 for buy, -1 for sell, 0 for hold).
    """
    df = df.copy()
    df['Signal'] = 0
    df.loc[df['GAPO'] < (threshold * 0.9), 'Signal'] = 1  # Buy signal
    df.loc[df['GAPO'] > (threshold * 1.1), 'Signal'] = -1  # Sell signal
    return df

def signal_to_action(signal: pd.Series) -> pd.DataFrame:
    """
    Convert trading signals into buy/sell action indicators.

    This function takes a signal series and generates a DataFrame with two columns:
    'buy' and 'sell'. A value of 1 in 'buy' indicates a buy action, while a value of -1
    in 'sell' indicates a sell action.

    Args:
        signal (pd.Series): A pandas Series containing trading signals (1 for buy, -1 for sell, 0 for hold).

    Returns:
        pd.DataFrame: A DataFrame with 'buy' and 'sell' columns indicating the respective actions.
    """
    columns = ['buy', 'sell']
    action = pd.DataFrame(np.zeros((signal.shape[0], 2)),
                          index=signal.index,
                          columns=columns,
                          dtype=int)
    action.loc[signal == 1, 'buy'] = 1
    action.loc[signal == -1, 'sell'] = -1
    return action

In [5]:
df_gapo = gapo(df)
df_gapo.dropna(inplace=True)
df_gapo = gapo_signal(df_gapo)

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_gapo.index, y=df_gapo['GAPO'], mode='lines', name='GAPO'))
fig.update_layout(title='Gopalakrishnan Range Index (GAPO)',
                  title_x=0.5,
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

## Know Sure Thing (KST)

In [7]:
def kst(df: pd.DataFrame, long_period: int = 34, short_period: int = 23, signal_period: int = 10) -> pd.DataFrame:
    """Calculates the Know Sure Thing (KST) indicator and its signal line.

    The KST indicator is a momentum oscillator that combines multiple rate-of-change (ROC) calculations.

    Args:
        df (pd.DataFrame): A DataFrame containing at least a 'Close' column with closing prices.
        long_period (int, optional): The long-period moving average window. Defaults to 34.
        short_period (int, optional): The short-period moving average window. Defaults to 23.
        signal_period (int, optional): The period for the KST signal line. Defaults to 10.

    Returns:
        pd.DataFrame: The input DataFrame with added 'KST' and 'KST_signal' columns.
    """
    df = df.copy()
    roc1 = df['Close'].pct_change(periods=10) * 100
    roc2 = df['Close'].pct_change(periods=15) * 100
    roc3 = df['Close'].pct_change(periods=20) * 100
    roc4 = df['Close'].pct_change(periods=30) * 100

    kst = (roc1.rolling(window=long_period).mean() +
           roc2.rolling(window=short_period).mean() +
           roc3.rolling(window=short_period).mean() +
           roc4.rolling(window=long_period).mean())

    df['KST'] = kst
    df['KST_signal'] = kst.rolling(window=signal_period).mean()
    return df

def generate_kst_signal(df: pd.DataFrame) -> pd.DataFrame:
    """Generates buy/sell/hold signals based on the KST indicator.

    The function assigns:
    - `1` (Buy) when KST crosses above KST_signal.
    - `-1` (Sell) when KST crosses below KST_signal.
    - `0` (Hold) otherwise.

    Args:
        df (pd.DataFrame): A DataFrame containing 'KST' and 'KST_signal' columns.

    Returns:
        pd.DataFrame: The input DataFrame with an added 'KST_signal_trade' column containing buy/sell/hold signals.
    """
    df = df.copy()
    df['KST_signal_trade'] = 0
    df.loc[df['KST'] > df['KST_signal'], 'Signal'] = 1   # Buy signal
    df.loc[df['KST'] < df['KST_signal'], 'Signal'] = -1  # Sell signal
    return df

In [8]:
df_kst = kst(df)
df_kst.dropna(inplace=True)
df_kst = generate_kst_signal(df_kst)

In [9]:
df_kst.head()

,Close,High,Low,Open,Volume,KST,KST_signal,KST_signal_trade,Signal
Date,,,,,,,,,
2020-04-16,260.282013,261.149302,257.167212,260.328630,131798300,-38.794218,-51.510079,0,1.0
2020-04-17,267.313629,267.929104,263.359487,266.138574,146684800,-34.222749,-49.236140,0,1.0
2020-04-20,262.604095,267.453502,262.380286,263.555313,100109300,-30.122639,-46.452283,0,1.0
2020-04-21,254.630646,259.293527,253.679400,258.071855,126385700,-25.999879,-43.322073,0,1.0
2020-04-22,260.282013,262.053902,258.239669,259.582581,92951600,-22.457624,-39.875219,0,1.0


In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_kst.index, y=df_kst['KST'], mode='lines', name='KST'))
fig.add_trace(go.Scatter(x=df_kst.index, y=df_kst['KST_signal'], mode='lines', name='KST Signal'))
fig.update_layout(title='Know Sure Thing (KST)',
                  title_x=0.5,
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

## Trend Intensity Index (TII)

In [15]:
def tii(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """Calculates the Trend Intensity Index (TII).

    The Trend Intensity Index measures the strength of a trend
    by comparing price differences over a given period to the rolling standard deviation.

    Args:
        df (pd.DataFrame): A DataFrame containing at least a 'Close' column with closing prices.
        period (int, optional): The lookback period for the TII calculation. Defaults to 14.

    Returns:
        pd.DataFrame: The input DataFrame with added 'trend' and 'trend_intensity' columns.
    """
    df = df.copy()
    df['trend'] = df['Close'] - df['Close'].shift(period)
    df['trend_intensity'] = df['trend'] / df['Close'].rolling(window=period).std()
    return df

def generate_tii_signal(df: pd.DataFrame, threshold: float = 1.0) -> pd.DataFrame:
    """Generates buy/sell/hold signals based on the Trend Intensity Index (TII).

    The function assigns:
    - `1` (Buy) when `trend_intensity` > `threshold`.
    - `-1` (Sell) when `trend_intensity` < `-threshold`.
    - `0` (Hold) otherwise.

    Args:
        df (pd.DataFrame): A DataFrame containing 'trend_intensity' column.
        threshold (float, optional): The intensity threshold for buy/sell signals. Defaults to 1.0.

    Returns:
        pd.DataFrame: The input DataFrame with an added 'TII_signal_trade' column containing buy/sell/hold signals.
    """
    df = df.copy()
    df['Signal'] = 0
    df.loc[df['trend_intensity'] > threshold, 'Signal'] = 1   # Buy signal
    df.loc[df['trend_intensity'] < -threshold, 'Signal'] = -1  # Sell signal
    return df


In [16]:
df_tii = tii(df)
df_tii.dropna(inplace=True)
df_tii = generate_tii_signal(df_tii)

In [17]:
df_tii.head()

,Close,High,Low,Open,Volume,trend,trend_intensity,Signal
Date,,,,,,,,
2020-01-23,307.545746,307.962963,305.404090,306.535184,51963000,6.350800,1.978549,1
2020-01-24,304.810760,308.296758,303.503511,308.213320,87578400,5.896606,2.032005,1
2020-01-27,299.924713,301.426650,299.145932,299.488964,84062500,-0.129822,-0.044519,0
2020-01-28,303.067719,303.957750,300.017471,301.371065,63834000,3.856873,1.496487,1
2020-01-29,302.817383,304.680912,302.613414,304.449131,53888900,2.011841,0.834320,0


In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_tii.index, y=df_tii['trend_intensity'], mode='lines', name='TII'))
fig.update_layout(title='Trend Intensity Index (TII)',
                  title_x=0.5,
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

## Backtesting

In [27]:
df_test = df_tii.copy()
actions = signal_to_action(df_test['Signal']).to_numpy()
entries = actions[:, 0]
exits = actions[:, 1]

In [28]:
signal_to_action(df_test['Signal']).head()

,buy,sell
Date,,
2020-01-23,1,0
2020-01-24,1,0
2020-01-27,0,0
2020-01-28,1,0
2020-01-29,0,0


In [29]:
portfolio = vbt.Portfolio.from_signals(
    close=df_test['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001,
    freq='D',
    # accumulate=True # this makes the backtest accumulate the position size
)

In [30]:
portfolio.stats()

Start                                2020-01-23 00:00:00
End                                  2024-12-31 00:00:00
Period                                1244 days 00:00:00
Start Value                                     100000.0
End Value                                  145577.995565
Total Return [%]                               45.577996
Benchmark Return [%]                           90.566777
Max Gross Exposure [%]                             100.0
Total Fees Paid                              6984.349684
Max Drawdown [%]                               19.047543
Max Drawdown Duration                  383 days 00:00:00
Total Trades                                          29
Total Closed Trades                                   29
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   48.275862
Best Trade [%]                                 18.205437
Worst Trade [%]                

In [31]:
fig = portfolio.plot_orders()
fig.update_layout(template='plotly_dark', width=1200)
fig.show()

In [32]:
fig = portfolio.plot_trade_pnl()
fig.update_layout(template='plotly_dark', width=1200)
fig.show()